In [1]:
#getting our own spotify data

In [2]:
#importing libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from skimage import io


In [3]:
#Authentication - without user using Spotify's Web API and making a new spotify developer account 
client_credentials_manager = SpotifyClientCredentials(
    client_id="14ab08163a0649be865dc476a3076fd7", client_secret="0ce7157e93604e008992ec4db58d8e70")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
#Getting user playlist
playlist_link = input("Enter your playlist link: ")
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"]
              for x in sp.playlist_tracks(playlist_URI)["items"]]



In [5]:
images=[]
for i in range(len(track_uris)):
    track = sp.track(track_uris[i])
    images.append(track["album"]["images"][0]["url"])

['https://i.scdn.co/image/ab67616d0000b273777d3075b2f6757b4808400b', 'https://i.scdn.co/image/ab67616d0000b273899243231842ed6dfa3186a1', 'https://i.scdn.co/image/ab67616d0000b273fa465afa3f4524c0dbb662da', 'https://i.scdn.co/image/ab67616d0000b2734aa295ee2ac086567c79bfc7', 'https://i.scdn.co/image/ab67616d0000b273ecdddbfde15bc99b5bedffd4', 'https://i.scdn.co/image/ab67616d0000b273dfa9036a64221e25554f3f21', 'https://i.scdn.co/image/ab67616d0000b273fa446c5e7de864d4c5b8a448', 'https://i.scdn.co/image/ab67616d0000b2731db2cbcfca93423a43f47c93', 'https://i.scdn.co/image/ab67616d0000b27352d4ece8c294aba86eac0e21', 'https://i.scdn.co/image/ab67616d0000b2738f030fcb66d5c65f8f510f14', 'https://i.scdn.co/image/ab67616d0000b273367ee242d8bada73b4630305', 'https://i.scdn.co/image/ab67616d0000b27370638b974587d549d6a2326a', 'https://i.scdn.co/image/ab67616d0000b27315c604efc4c884aeb42035d7', 'https://i.scdn.co/image/ab67616d0000b2735c30005b07efd927287a1bea', 'https://i.scdn.co/image/ab67616d0000b2732f1db5

In [6]:
#Acosticness confidence measure from 0.0 to 1.0 of whether the track is acoustic.
#Danceability measure describes how suitable a track is for dancing.
#duration_ms is the duration of the song track in milliseconds.
#Energy represents a perceptual measure of intensity and activity.
#Instrumentalness predicts whether a track contains vocals or not.
#Loudness of a track in decibels(dB).
#Liveness detects the presence of an audience in the recording.
#Speechiness detects the presence of spoken words in a track
#Time_signature is an estimated overall time signature of a track.
#Key the track is in. Integers map to pitches using standard Pitch Class notation.
#Valence measures from 0.0 to 1.0 describing the musical positiveness conveyed by a track.
#Target value describes the encoded value of 0 and 1. 0 means listener has not saved the song and 1 means listener have saved the song.
#Tempo is in beats per minute(BPM).
#Mode indicates the modality(major or minor) of the song.
#Song_title is the name of the song.
#Artist is the singer of the song.
#URL is the link to the song.



In [7]:
#making a new csv file with th data and headers as the features of the audio features and also Song Name and Artist Name
song_name = []
artist_name = []
acousticness = []
danceability = []
duration_ms = []
energy = []
instrumentalness = []
loudness = []
liveness=[]
speechiness = []
time_signature = []
key = []
valence = []
tempo = []
mode = []
target = []
url = []
#get track name and artist name
for track_uri in track_uris:
    track = sp.track(track_uri)
    song_name.append(track["name"])
    artist_name.append(track["artists"][0]["name"])
    audio_features = sp.audio_features(track_uri)
    acousticness.append(audio_features[0]["acousticness"])
    danceability.append(audio_features[0]["danceability"])
    duration_ms.append(audio_features[0]["duration_ms"])
    energy.append(audio_features[0]["energy"])
    instrumentalness.append(audio_features[0]["instrumentalness"])
    loudness.append(audio_features[0]["loudness"])
    liveness.append(audio_features[0]["liveness"])
    speechiness.append(audio_features[0]["speechiness"])
    time_signature.append(audio_features[0]["time_signature"])
    key.append(audio_features[0]["key"])
    valence.append(audio_features[0]["valence"])
    tempo.append(audio_features[0]["tempo"])
    mode.append(audio_features[0]["mode"])
    target.append(1)
    url.append(track["external_urls"]["spotify"])
#make a csv file with the data
df = pd.DataFrame({'Song Name':song_name, 'Artist Name':artist_name, 'Acousticness':acousticness, 'Danceability':danceability, 'Duration_ms':duration_ms, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Loudness':loudness,'Liveness':liveness ,'Speechiness':speechiness, 'Time_signature':time_signature, 'Key':key, 'Valence':valence, 'Tempo':tempo, 'Mode':mode, 'Target':target, 'URL':url, 'Image':images})
df.to_csv('audio_features.csv', index=False)


In [8]:
df = pd.read_csv("Audio_features.csv")
df.head()

,Song Name,Artist Name,Acousticness,Danceability,Duration_ms,Energy,Instrumentalness,Loudness,Liveness,Speechiness,Time_signature,Key,Valence,Tempo,Mode,Target,URL,Image
0,"Dil Di Dua (From ""Bhalwan Singh"" Soundtrack)",Amrinder Gill,0.130,0.497,173041,0.640,0.000000,-6.488,0.1130,0.0786,3,9,0.643,150.096,1,1,https://open.spotify.com/track/55TWWMXWUNjdDeS...,https://i.scdn.co/image/ab67616d0000b273777d30...
1,"Saah (From ""Lahoriye"" Soundtrack)",Bir Singh,0.889,0.229,356610,0.148,0.000032,-12.954,0.1120,0.0398,4,3,0.323,57.861,1,1,https://open.spotify.com/track/3kxyibOgYLPY7Qc...,https://i.scdn.co/image/ab67616d0000b273899243...
2,"Jeeondean Ch (From ""Lahoriye"" Soundtrack)",Amrinder Gill,0.672,0.710,253061,0.628,0.000000,-6.259,0.1650,0.0367,4,11,0.659,114.897,0,1,https://open.spotify.com/track/5INxn9kacljD6WO...,https://i.scdn.co/image/ab67616d0000b273fa465a...
3,"Sukh Da Saah (From ""Vekh Baraatan Challiyan"" S...",Amrinder Gill,0.578,0.551,214759,0.628,0.000001,-5.641,0.1240,0.0539,4,10,0.314,125.427,1,1,https://open.spotify.com/track/6TwTVytvcZezbyO...,https://i.scdn.co/image/ab67616d0000b2734aa295...
4,Mil Ke Baithange,Amrinder Gill,0.710,0.754,218941,0.559,0.000000,-7.957,0.0996,0.0318,1,4,0.326,112.057,1,1,https://open.spotify.com/track/3b5nizka1oGpQiO...,https://i.scdn.co/image/ab67616d0000b273ecdddb...


In [9]:
from sklearn.preprocessing import MinMaxScaler
feature_cols = ['Acousticness', 'Danceability', 'Duration_ms', 'Energy',
                'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode',
                'Speechiness', 'Tempo', 'Time_signature', 'Valence', ]

scaler = MinMaxScaler()
normalized_df = scaler.fit_transform(df[feature_cols])

print(normalized_df[:2])


[[1.13871636e-01 5.10476190e-01 4.45294325e-02 6.73972603e-01
  0.00000000e+00 8.18181818e-01 1.96597353e-01 7.05109086e-01
  1.00000000e+00 1.56107911e-01 9.06967826e-01 6.66666667e-01
  6.30000000e-01]
 [8.99585921e-01 0.00000000e+00 6.03488910e-01 0.00000000e+00
  1.04516129e-04 2.72727273e-01 1.94234405e-01 3.47970174e-01
  1.00000000e+00 3.84965141e-02 0.00000000e+00 1.00000000e+00
  2.30000000e-01]]


In [21]:
# Create a pandas series with song titles as indices and indices as series values
indices = pd.Series(df.index, index=df['Song Name']).drop_duplicates()

# Create cosine similarity matrix based on given matrix
cosine = cosine_similarity(normalized_df)


def generate_recommendation(song_title,limit_songs, model_type=cosine):
    """
    Purpose: Function for song recommendations 
    Inputs: song title and type of similarity model
    Output: Pandas series of recommended songs
    """
    # Get song indices
    index = indices[song_title]
    global song_image
    
    # Get list of songs for given songs
    score = list(enumerate(model_type[indices[song_title]]))
    # Sort the most similar songs
    similarity_score = sorted(score, key=lambda x: x[1], reverse=True)
    # Select the top-user input recommended songs
    similarity_score = similarity_score[1:limit_songs+1]
    top_songs_index = [i[0] for i in similarity_score]
    # Top user-input value recommended songs
    top_songs = df['Song Name'].iloc[top_songs_index]
    top_songs_artists = df['Artist Name'].iloc[top_songs_index]
    song_image=[df['Image'].iloc[i] for i in top_songs_index]
    result_frame=pd.DataFrame({'Song Name':top_songs, 'Artist Name':top_songs_artists})
    return result_frame
    


In [22]:
#Print all the songs in the playlist
for i in range(len(song_name)):
    print(song_name[i],":",artist_name[i])


Dil Di Dua (From "Bhalwan Singh" Soundtrack) : Amrinder Gill
Saah (From "Lahoriye" Soundtrack) : Bir Singh
Jeeondean Ch (From "Lahoriye" Soundtrack) : Amrinder Gill
Sukh Da Saah (From "Vekh Baraatan Challiyan" Soundtrack) : Amrinder Gill
Mil Ke Baithange : Amrinder Gill
Do Gallan - Let's Talk : Garry Sandhu
Akhar (From "Lahoriye" Soundtrack) : Amrinder Gill
Dholna : B Praak
Channa Ve : B Praak
Masoomiat : Satinder Sartaaj
Supna : Amrinder Gill
Jinne Saah : Ninja
Sun Sohniye - From "Afsar" : Ranjit Bawa
Rabb Khair Kare (From "Daana Paani" Soundtrack) : Prabh Gill
Chunni (From "Lahoriye" Soundtrack) : Amrinder Gill
Rubaru (From "Saadi Love Story") : Amrinder Gill
Beet Jaania'N (Endless Love) : Satinder Sartaaj
Meri Heeriye Fakiriye : Satinder Sartaaj
Chan Kithe Guzari Ayeerat : Surinder Kaur
Akhian Ch Tu Vasda : Surinder Kaur
Leekan (From "Ashke" Soundtrack) : Amrinder Gill
Ek Meri Akh Kashni - Punjabi Geet : Surinder Kaur
Channa Ve : B Praak
Udaarian : Satinder Sartaaj
Asi Gabru Punjabi

In [23]:
song_value=input("Write the song name you want to recommend from this playlist")

KeyboardInterrupt: Interrupted by user

In [ ]:
number=int(input("Number of Recommended Songs you want to see"))  

In [ ]:
print("Recommended Songs:")
print(generate_recommendation(song_value,number,cosine))

Recommended Songs:
                                    Song Name       Artist Name
14        Chunni (From "Lahoriye" Soundtrack)     Amrinder Gill
24                          Asi Gabru Punjabi     Amrinder Gill
2   Jeeondean Ch (From "Lahoriye" Soundtrack)     Amrinder Gill
23                                   Udaarian  Satinder Sartaaj
12                 Sun Sohniye - From "Afsar"       Ranjit Bawa


In [ ]:
#show all the images of the recommended songs
for i in range(len(song_image)):
    image=io.imread(song_image[i])
    plt.imshow(image)
    plt.xlabel(song_name[i],size=10)